# Steam Game Recommendation

This is the first notebook in the series to build a steam game recommendation engine based on game tags. This notebook focuses on obtaining the data.

## Downloading Data

In [1]:
import json
import os
import time

In [2]:
os.chdir('..')

In [3]:
import download

Becuse the API call to obtain all the data does not return game tags, we make an additional call for each game in the dataset to obtain the tags. Note, that steamspy will throttle us to 4 calls per second.

In [4]:
tag_games = os.listdir('./data')

In [12]:
with open('game_tags.txt', 'r') as tags_in:
    for i, tag in enumerate(tags_in):
        tag_filename = tag.strip().replace(' ', '_')
        file_out = f'{tag_filename}_games.json'
        if file_out in tag_games:
            with open(f'./data/{file_out}', 'r') as json_in:
                tag_data = json.load(json_in)
                if tag_data:
                    continue
        tag_data = download.get_games_with_tag(tag.strip())
        tag = tag.strip().replace(' ', '_')
        with open(f'./data/{tag}_games.json', 'w') as json_out:
            json.dump(tag_data, json_out)
        if i % 100 == 0:
            print(i)
        time.sleep(0.250) # Throttled to 4 requests a second

300
400


Finally, write the data set to a json file